In [1]:
from bibleabbr import *

In [2]:
import numpy as np
import shorttext
import sqlite3

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Scripture Loaded

In [3]:
dbconn = sqlite3.connect('ESVCrossway.sqlite')

In [4]:
from functools import reduce
from nltk import sent_tokenize

def retrieve_docs_as_biblechapters(dbconn):
    cursor = dbconn.cursor()
    for book in wholebible_book_iterator():
        for chap in range(1, numchaps[book]+1):
            doc_label = book+'_'+str(chap)
            result = cursor.execute('select scripture from bible where book is "'+book+'" and chapter='+str(chap))
            doc_text = reduce(lambda s1, s2: ' '.join([s1, s2]), [texttuple[0] for texttuple in result])
            yield doc_label, doc_text
    cursor.close()
    

In [5]:
from collections import defaultdict

classdict = defaultdict(lambda : [])
for bible_chap, verses in retrieve_docs_as_biblechapters(dbconn):
    for verse in sent_tokenize(verses):
        classdict[bible_chap] += [verse]

# C-LSTM

In [6]:
wmodel = shorttext.utils.load_word2vec_model('wordembedD/wiki-news-300d-1M.vec', binary=False)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
kmodel = shorttext.classifiers.CNNWordEmbed(len(classdict.keys()), vecsize=wmodel.vector_size)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wmodel)

In [9]:
classifier.train(classdict, kmodel)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
35885/35885 [==============================] - 54s 2ms/step - loss: 6.1203
Epoch 2/10
35885/35885 [==============================] - 53s 1ms/step - loss: 4.0913
Epoch 3/10
35885/35885 [==============================] - 53s 1ms/step - loss: 2.8235
Epoch 4/10
35885/35885 [==============================] - 54s 1ms/step - loss: 1.8617
Epoch 5/10
35885/35885 [==============================] - 53s 1ms/step - loss: 1.1644
Epoch 6/10
35885/35885 [==============================] - 53s 1ms/step - loss: 0.7227
Epoch 7/10
35885/35885 [==============================] - 53s 1ms/step - loss: 0.4709
Epoch 8/10
35885/35885 [==============================] - 52s 1ms/step - loss: 0.3421
Epoch 9/10
35885/35885 [==============================] - 53s 1ms/step - loss: 0.2778
Epoch 10/10
35885/35885 [==============================] - 35s 978us/step - loss: 0.2458


In [10]:
classifier.save_compact_model('biblechap_CNN_ft.pkl')

# Prediction

In [4]:
import shorttext

wmodel = shorttext.utils.load_word2vec_model('wordembedD/wiki-news-300d-1M.vec', binary=False)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
classifier = shorttext.smartload_compact_model('biblechap_CNN_ft.pkl', wmodel)

In [12]:
sorted(classifier.score('justice').items(), key=lambda item: item[1], reverse=True)[:10]

[('la_4', 0.116194434),
 ('ps_99', 0.10750595),
 ('ps_37', 0.064568534),
 ('pr_21', 0.052186467),
 ('mic_3', 0.050938107),
 ('pr_2', 0.030844092),
 ('ps_72', 0.027011933),
 ('is_42', 0.026015548),
 ('zep_3', 0.02564504),
 ('is_43', 0.023865286)]